In [1]:
import pandas as pd
import re

In [2]:
file_path = 'network_expansion2.xlsx'
sim_df = pd.read_excel(file_path, sheet_name='TableS1')
gen_df = pd.read_excel(file_path, sheet_name='TableS5').set_index("compound_id")

In [3]:
gen_df.modified = gen_df.modified.astype('float32')

In [4]:
gen_mapper = gen_df['modified'].to_dict()

In [5]:
sim_df.columns = ['reaction', 'direction', 'source', 'coenzyme', 'equation']

In [6]:
sim_df.loc[:, "reactants"] = sim_df.equation.str.split(" => ", expand=True)[0]
sim_df.loc[:, "products"]  = sim_df.equation.str.split(" => ", expand=True)[1]

In [7]:
def extract_compounds(st):
    all_compounds = re.findall("([CZ][0-9]{5})", st)
    return all_compounds

def max_gen(compounds, gen_mapper):
    all_gens = list(map(lambda x: gen_mapper.get(x, -1.0), compounds))
    
    if len(all_gens) > 0:
        max_compound_gen = max(all_gens)
    else:
        max_compound_gen = -1.0 
    return max_compound_gen

def map_generation(st, gen_mapper):
    compounds = extract_compounds(st)
    gen = max_gen(compounds, gen_mapper)
    return gen 

map_generation("C01839", gen_mapper)

-1.0

In [8]:
sim_df.loc[:, "reactant_gen"] = sim_df.reactants.apply(lambda x: map_generation(x, gen_mapper))
sim_df.loc[:, "product_gen"]  = sim_df.products.apply(lambda x: map_generation(x, gen_mapper))

In [23]:
sim_df = sim_df[(sim_df.product_gen >=  sim_df.reactant_gen) & (sim_df.product_gen >= 0) & (sim_df.reactant_gen >= 0)]

In [24]:
sim_df.loc[:, "generation"] = (sim_df.reactant_gen + sim_df.product_gen) / 2

In [27]:
sim_df.generation.max()

99.5

In [29]:
ec_df = pd.read_csv('simulations.csv')
ec_df = ec_df[~ec_df.ec_list.isna()][['reaction', 'ec_list']]
ec_df.head()

,reaction,ec_list
12,R00636,4.1.1.1
13,R00636,4.1.1.1
14,R00217_v1,"1.1.1.38, 1.1.1.40, 4.1.1.112"
15,R00217_v1,"1.1.1.38, 1.1.1.40, 4.1.1.112"
16,R00217_v2,"1.1.1.38, 1.1.1.40, 4.1.1.112"


In [32]:
df = pd.merge(sim_df, ec_df, on="reaction", how='left')
df = df.sort_values("generation")

In [33]:
df.to_csv('simulations.csv')